<a href="https://colab.research.google.com/github/arqPy/Instala-es_El-tricas/blob/main/C%C3%A1lculos_Instala%C3%A7%C3%B5es_El%C3%A9tricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Diagrama de Classes**



<p align="center">
  <img width="3000" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/Diagrama_de_Classes.png?raw=true">
</p>

Grafo da Instalação

<p align="center">
  <img width="350" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/Graph_Installation.png?raw=true">
</p>



Vértice (Carga Elétrica)
<p align="center">
  <img width="200" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/Carga_Eletrica.png?raw=true">
</p>



Arco (Eletroduto - Conduite)

<p align="center">
  <img width="250" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/Conduite.png?raw=true">
</p>

In [ ]:
class electricLoad: # Vertices do Grafo
  def __init__(self, pot, comodo, circ= 0, x=0, y=0, z=0):
    self.Pot = pot
    self.comodo = comodo
    self.Circ = circ
    #self.QD = 0
    self.x = x
    self.y = y
    self.z = z

  def potencia(self):
    return self.Pot

  def Comodo(self):
    return self.comodo

  def setCirc(self, idCirc):
    self.Circirc = idCirc

  def getCirc(self):
    return self.Circ

  '''
  def setQD(self, idQD):
    self.QD = idQD

  def getQD(self):
    return self.QD

  '''

class lamp(electricLoad):
  def __init__(self, pot, comodo):
    super().__init__(pot, comodo)

  def potencia(self):
    return self.Pot

  def Comodo(self):
    return self.comodo

class QD(electricLoad):
  def __init__(self, pot, comodo):
    super().__init__(pot, comodo)
    self.Circuitos = {}
    self.Tensao = 220

  def idCircuitos(self):
    return self.Circuitos.keys()

  def potencia(self): # potencia total do quadro de distribuicao
    return self.Pot

  def listCircuits(self, id): # de um dado circuito
    return self.Circuitos[id]

  def createCircuito(self, idCirc, lLoadsCirc, dictLoads, Smin=2.5):
      pCirc = 0
      for idload in lLoadsCirc:
        L = dictLoads[idload]
        if (L.getCirc() == 0):
          pCirc = pCirc + L.potencia()
          L.setCirc(idCirc)

      C = Circuito(idCirc, lLoadsCirc, pCirc, Smin)
      self.Circuitos[idCirc] = C

  def Comodo(self):
    return self.comodo


class TUE(electricLoad):
  def __init__(self, pot, comodo):
    super().__init__(pot, comodo)

  def potencia(self):
    return self.Pot

  def Comodo(self):
    return self.comodo

class electricOutlet(electricLoad):
  def __init__(self, pot, comodo):
    super().__init__(pot, comodo)

  def potencia(self):
    return self.Pot

  def Comodo(self):
    return self.comodo

class Circuito:
  def __init__(self, idCirc, loadsCirc, potCirc, Smin=2.5):
    self.loads = loadsCirc
    self.idCirc = idCirc
    #self.Tensao = Tensao
    self.potCirc = potCirc
    self.Smin = Smin

  def potCircuit(self):
    return self.potCirc

  def setPotCircuit(self, pot):
    self.potCirc = pot

  def setLoadsCircuit(self, loadsCirc):
    self.loads = loadsCirc

  def listLoadsCircuit(self):
    return self.loads

class graphInstallation:  #Conjunto de Vertices e Arcos
  def __init__(self, arquitetura):
    self.arquitetura = arquitetura
    self.Loads = {} # dicionário para referenciar diretamente cada carga - VERTICES
    self.QDs = {} # pode criar um dicionario de QDs {}. Considerando apenas UM
    #self.Circuitos = {}
    self.TA = 'M1'
    self.Pinst = 0
    self.__addLoad()

  def __potIlum(self, comodo, area):
    p = 100
    if area < 6:
      return p
    d = (area - 6) // 4
    return p + d*60

  def createQD(self, idQD):
    qd = QD(self.Pinst, "Residencia")
    self.QDs[idQD] = qd
    self.Loads["QD"+ str(idQD)] = qd

  def getQDs(self):
    return self.QDs

  def __qtdTug(self, comodo, area, per):
    q100, q600 = 1, 0

    if (comodo == 'Banheiro'):
      q100, q600 = 0, 1

    if (comodo == 'Sala') or (comodo == 'Quarto'):
      q100 = int(1 + per // 5)

    if ((comodo == 'Área de Serviço') or (comodo == 'Cozinha')):
      q600 = int(1 + per // 3.5)
      q100 = 0 # pode retirar?
      if q600 > 2:
        q100 = q600 - 3
        q600 = 3
    return (q100, q600)

  def listLoads(self):
    return self.Loads


  def addTUEs(self, listTues):
    ids = listTues.keys()
    k = 0
    for id in ids:
      E = TUE(listTues[id]['Pot'], listTues[id]['Comodo'])
      k= k + 1
      key = "E" + str(k)
      #print(id, key, Pot)
      self.Loads[key] = E

  def __addLoad(self):
    ids = self.arquitetura.keys()
    k = 0
    for id in ids:
      ele = self.arquitetura[id]
      area= ele['C']*ele['L']
      per = 2*(ele['C'] + ele['L'])

      Pot = self.__potIlum(id, area) # Potencia de Iluminação
      L = lamp(Pot, id)
      k= k + 1
      key = "L" + str(k)
      #print(id, key, Pot)
      self.Loads[key] = L

      (qTUG100, qTUG600) = self.__qtdTug(id, area, per)

      for i in range(qTUG100):
        R = electricOutlet(100, id)
        k= k + 1
        key = "R" + str(k)
        self.Loads[key] = R

      for i in range(qTUG600):
        R = electricOutlet(600, id)
        k= k + 1
        key = "R" + str(k)
        self.Loads[key] = R

  def potenciaInstalada(self):
      loads = self.Loads
      ids = loads.keys()
      pIlum, pTug, pTue = 0, 0, 0
      for id in ids:
        l = loads[id]
        pot = l.potencia()
        if id[0:1]== 'L':
          pIlum = pIlum + pot
        elif id[0:1]== 'R':
          pTug = pTug + pot*0.8 # fator de potencia das TUGs
        elif id[0:1]== 'E':
          pTue = pTue + pot
      Pinst = pIlum + pTug + pTue
      TA = 'M1'
      if ((Pinst > 15000)and (Pinst < 22000)):
        TA = 'B2'
      if ((Pinst > 11000)and (Pinst < 15000)):
        TA = 'B1'
      if ((Pinst > 8000)and (Pinst < 11000)):
        TA = 'M2'
      if (Pinst > 22000):
        TA = 'T'
      self.Pinst = Pinst
      self.TA = TA
      return (Pinst, TA)



In [ ]:
from urllib.request import urlopen

import json

url = "https://raw.githubusercontent.com/arqPy/Instala-es_El-tricas/main/fig/arquitetura03.json"

response = urlopen(url)

data_json = json.loads(response.read())

arquitetura = {}
for item in data_json:
  key = item['Comodo']
  value = item['Dimensões']
  arquitetura[key] = value
print(arquitetura)
tues = {
    'Chuveiro_B1':{'Pot':6500,"Comodo": 'Banheiro'}
}


{'Área de Serviço': {'C': 1.75, 'L': 3.4}, 'Dormitório B': {'C': 3.15, 'L': 3.4}, 'Dormitório A': {'C': 3.25, 'L': 3.4}, 'Sala': {'C': 3.25, 'L': 3.05}, 'Copa': {'C': 3.1, 'L': 3.05}, 'Cozinha': {'C': 3.75, 'L': 3.05}, 'Hall': {'C': 1.8, 'L': 1}, 'Banheiro': {'C': 1.8, 'L': 2.3}}


In [ ]:
projInst = graphInstallation(arquitetura)
projInst.addTUEs(tues)
print(projInst.potenciaInstalada())
projInst.createQD(1)
print("QD:",projInst.getQDs())
qd = projInst.getQDs()[1]
print("Pot=", qd.potencia())
loads = projInst.listLoads()
for lo in loads.keys():
  print(lo)

(11480.0, 'B1')
QD: {1: <__main__.QD object at 0x7c4603256e30>}
Pot= 11480.0
L1
R2
R3
R4
L5
R6
L7
R8
L9
R10
R11
R12
L13
R14
L15
R16
R17
R18
R19
L20
R21
L22
R23
E1
QD1


In [ ]:
listInst = loads.keys()
listIlu = [x for x in listInst if x[0]  == "L"]
listForce = [x for x in listInst if x[0]  != "L"]
for id in listIlu:
  oneLoad = loads[id]
  print(id, oneLoad.Comodo(), oneLoad.potencia())

for id in listForce:
  oneLoad = loads[id]
  print(id, oneLoad.Comodo(), oneLoad.potencia())


L1 Área de Serviço 100
L5 Dormitório B 160.0
L7 Dormitório A 160.0
L9 Sala 100.0
L13 Copa 100.0
L15 Cozinha 160.0
L20 Hall 100
L22 Banheiro 100
R2 Área de Serviço 600
R3 Área de Serviço 600
R4 Área de Serviço 600
R6 Dormitório B 100
R8 Dormitório A 100
R10 Sala 100
R11 Sala 100
R12 Sala 100
R14 Copa 100
R16 Cozinha 100
R17 Cozinha 600
R18 Cozinha 600
R19 Cozinha 600
R21 Hall 100
R23 Banheiro 600
E1 Banheiro 6500
QD1 Residencia 11480.0


Exemplo

<p align="center">
  <img width="800" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/planta_arq.png?raw=true">
</p>




<p align="center">
  <img width="600" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/CargasIluminac%CC%A7a%CC%83o.png?raw=true">
</p>

L1 Área de Serviço 100
L5 Dormitório B 160.0
L7 Dormitório A 160.0
L9 Sala 100.0
L13 Copa 100.0
L15 Cozinha 160.0
L20 Hall 100
L22 Banheiro 100

<p align="center">
  <img width="600" src="https://github.com/arqPy/Instala-es_El-tricas/blob/main/fig/TUGs.png?raw=true">
</p>


R2 Área de Serviço 600
R3 Área de Serviço 600
R4 Área de Serviço 600
R6 Dormitório B 100
R8 Dormitório A 100
R10 Sala 100
R11 Sala 100
R12 Sala 100
R14 Copa 100
R16 Cozinha 100
R17 Cozinha 600
R18 Cozinha 600
R19 Cozinha 600
R21 Hall 100
R23 Banheiro 600